# Make Resnet

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import tqdm
import time
from sklearn.metrics import accuracy_score, recall_score

### 1. Data preprocessing

In [8]:
BATCH_SIZE = 100
LAYER_NM = 18
path_data = './data'

if not os.path.exists(path_data):
    os.mkdir(path_data)

transform_ = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x/255.0 for x in [63.0, 62.1, 66.7]])
])
train_data = datasets.STL10(root=path_data, split='train', download=True, transform=transform_)
test_data = datasets.STL10(root=path_data, split='test', download=True, transform=transform_)

train_loader = DataLoader(train_data,batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_data,batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=True)

# define classes
classes = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [9]:
print(next(iter(train_data))[0].size())
print(next(iter(train_loader))[0].size())

torch.Size([3, 128, 128])
torch.Size([100, 3, 128, 128])


### 2. Build Model

In [10]:
class BasicBlockLower(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlockLower, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                            stride=stride, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [11]:
class BasicBlockUpper(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlockUpper, self).__init__()
        
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3,
                            stride=stride, padding=1, bias=False)
        
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [12]:
class ResNet(nn.Module):
    def __init__(self, num_blocks, name, debugging=False, num_classes=10):
        super(ResNet, self).__init__()
        self.name = name
        self.debugging = debugging
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=2, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)
        self.flatten = nn.Flatten()

    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            if self.name == 'resnet18':
                layers.append(BasicBlockLower(self.in_planes, planes, stride))
            else:
                layers.append(BasicBlockUpper(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        if self.debugging is True:
            print('x:', x.size())
        out = F.max_pool2d(F.relu(self.bn1(self.conv1(x))), kernel_size=3, stride=2, padding=1)
        if self.debugging is True:
            print('start:', out.size())
        out = self.layer1(out)
        if self.debugging is True:
            print('layer1 out:', out.size())
        out = self.layer2(out)
        if self.debugging is True:
            print('layer2 out:', out.size())
        out = self.layer3(out)
        if self.debugging is True:
            print('layer3 out:', out.size())
        out = self.layer4(out)
        if self.debugging is True:
            print('layer4 out:', out.size())
        out = F.avg_pool2d(out, 4)
        if self.debugging is True:
            print('avg_pool out:', out.size())
        # out = out.view(out.size(0), -1)
        out = self.flatten(out)
        if self.debugging is True:
            print('falten out:', out.size())
        out = self.linear(out)
        return out

### Resnet 18 <br/>
x: torch.Size([100, 3, 128, 128]) <br/>
start: torch.Size([100, 64, 128, 128]) <br/>
layer1 out: torch.Size([100, 64, 128, 128]) <br/>
layer2 out: torch.Size([100, 128, 64, 64]) <br/>
layer3 out: torch.Size([100, 256, 32, 32]) <br/>
layer4 out: torch.Size([100, 512, 16, 16]) <br/>
avg_pool out: torch.Size([100, 512, 2, 2]) <br/>
falten out: torch.Size([100, 2048])

### 3. Train

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 1e-2
model_d = next(iter(train_loader))[0].size()[-2] * next(iter(train_loader))[0].size()[-1]
block_list = {'resnet18':[2,2,2,2], 'resnet34':[3,4,6,3], 'resnet50':[3,4,6,3], 
              'resnet101':[3,4,23,3], 'resnet152':[3,8,36,3]}
y_dim = 10 # output class
name = 'resnet18'
model = ResNet(block_list[name], name, debugging=False).to(device)
# model = ResNet(model_d, y_dim).to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

df_loss = pd.DataFrame(columns=['loss'])
epochs = 300
batch_size = len(train_loader)
for i in range(epochs):
    model.train()
    running_loss = 0.
    for image, label in train_loader:
        x = image.to(device)
        y = label.to(device)

        optimizer.zero_grad()
        output = model(x)
        loss = loss_func(output, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    df_loss.loc[i, 'loss'] = round(running_loss/batch_size, 4)
    print(f'epoch: {i}, Loss:', round(running_loss/batch_size, 7))
    scheduler.step()

epoch: 0, Loss: 2.4557535
epoch: 1, Loss: 1.8304498
epoch: 2, Loss: 1.6796636
epoch: 3, Loss: 1.564509
epoch: 4, Loss: 1.5036189
epoch: 5, Loss: 1.4410479
epoch: 6, Loss: 1.3549792
epoch: 7, Loss: 1.2628156
epoch: 8, Loss: 1.1882302
epoch: 9, Loss: 1.1111456
epoch: 10, Loss: 1.0045026
epoch: 11, Loss: 0.9176197
epoch: 12, Loss: 0.8018226
epoch: 13, Loss: 0.6786485
epoch: 14, Loss: 0.5572389
epoch: 15, Loss: 0.446047
epoch: 16, Loss: 0.3560451
epoch: 17, Loss: 0.2670573
epoch: 18, Loss: 0.2048811
epoch: 19, Loss: 0.1880614
epoch: 20, Loss: 0.1442785
epoch: 21, Loss: 0.0983283
epoch: 22, Loss: 0.0984912
epoch: 23, Loss: 0.0559359
epoch: 24, Loss: 0.0487518
epoch: 25, Loss: 0.0604478
epoch: 26, Loss: 0.0827655
epoch: 27, Loss: 0.05106
epoch: 28, Loss: 0.0320918
epoch: 29, Loss: 0.0214388
epoch: 30, Loss: 0.0531115
epoch: 31, Loss: 0.0452004
epoch: 32, Loss: 0.0479321
epoch: 33, Loss: 0.0369598
epoch: 34, Loss: 0.0199866
epoch: 35, Loss: 0.0128721
epoch: 36, Loss: 0.025162
epoch: 37, Loss:

In [14]:
df_loss

,loss
0,2.4558
1,1.8304
2,1.6797
3,1.5645
4,1.5036
...,...
295,0.0001
296,0.0001
297,0.0001
298,0.0001


In [15]:
correct = 0
total = 0
test_result = pd.DataFrame(columns=['output', 'y'])
with torch.no_grad():
    model.eval()
    for i, (input, label) in enumerate(test_loader):
        x = input.to(device)
        y = label.to(device)
        outputs = model(x)
        # value, pred = torch.max(outputs, 1)
        pred = torch.argmax(outputs, 1)
        out = pd.DataFrame(np.array([pred.detach().cpu().numpy(), y.detach().cpu().numpy()]).T, columns=['output', 'y'])
        # print(out)
        test_result = pd.concat([test_result, out], axis=0).reset_index(drop=True)

        total += y.size(0)
        correct += (torch.argmax(outputs, 1) == y).sum().item()
        
    print("test accuracy : {}%".format((100 * correct / total)))

test accuracy : 62.75%


In [17]:
test_result

,output,y
0,6,6
1,6,7
2,3,5
3,0,0
4,3,3
...,...,...
7995,2,9
7996,6,6
7997,8,8
7998,9,8


In [22]:
test_result['y'].values

array([6, 7, 5, ..., 8, 8, 8], dtype=object)

In [27]:
acc = accuracy_score(list(test_result['y'].values), list(test_result['output'].values))
rcll = recall_score(list(test_result['y'].values), list(test_result['output'].values), average='micro')
print('Accuracy:', acc, '  Recall:', rcll)

Accuracy: 0.6275   Recall: 0.6275
